In [1]:
#imports
import pandas as pd
import numpy as np
from pylab import *
import string
from unidecode import unidecode

#load csv
veg_data_to_load = "zomato.csv"
veg_data = pd.read_csv(veg_data_to_load, encoding='ISO-8859-1')
veg_data_df = pd.DataFrame(veg_data).reset_index()



In [2]:
# Creating the dataframe from desired columns
veg_data_df = veg_data_df[["Restaurant Name","Aggregate rating","Address", "City", "Votes"]]

#Cleaning the data.  Removing non-alphanumeric columns.  Don't include spaces
veg_data_df["Restaurant Name"] = veg_data_df["Restaurant Name"].str.replace('[^a-zA-Z ]', '')
#Cleaning the data.  Removing non-alphanumeric columns.  Don't include commas or spaces
veg_data_df["Address"] = veg_data_df["Address"].str.replace('[^a-zA-Z],', '')
#Cleaning the data.  Removing non-alphanumeric columns.  Don't include spaces
veg_data_df["City"] = veg_data_df["City"].str.replace('[^a-zA-Z] ', '')

veg_data_df = veg_data_df.rename(columns = {"Aggregate rating":"Rating"})

veg_data_df = veg_data_df.rename(columns = {"Votes":"Reviewers Count"})



In [3]:
#BRING IN YELP

#import
import requests
import pandas as pd
 
# Developer API key
API_KEY= "mk-Wsg9uQeXyKH3sa02giTl_xVqw3pC3cDNZna5OYlYPz27WIcX24kOxWXYD6d3XnQvqlOvGrZvCwTpC5Wl1jUZklx6ZX0q3FOqOs_KZ5wjLpHZC3u1XQk2ZLRRmXHYx" 
 
# What you are searching for
DEFAULT_TERM = 'Vegetarian'
# Business location

#Yelp has a bug that will only return 50 records at a time.
# to work around this, I am running once with the original list definitions,
# then appending.  I am using the following cities/locations:
# Los Angeles, San Francisco, New York, Seattle, Austin, Chicago, Portland, Pittsburgh, Albany, St. Louis, Minneapolis,
# Washington, Houston, Pennsylvania, San Diego, Houston, Phoenix, San Jose, Boston, Baltimore, Fresno, Florida 

DEFAULT_LOCATION = 'Los Angeles'
# Maximum number of results to return
SEARCH_LIMIT = 50
OFFSETN = 51
 
# Busines search end point
url = 'https://api.yelp.com/v3/businesses/search'
# Header should contain the API key
headers = {'Authorization': 'Bearer {}'.format(API_KEY)}
# Search parameters
url_params = {
	'term': DEFAULT_TERM, 
	'location': DEFAULT_LOCATION, 
	'limit': SEARCH_LIMIT,
    'offset': OFFSETN
	}


In [4]:
### Comment out all below after first run so list doesn't recreate itself.
all_rest = []
all_add = []
all_city = []
all_rating = []
all_review = []

In [5]:


# Call the API
response = requests.request('GET', url, headers=headers, params=url_params)
 
for business in response.json()["businesses"]:
    all_rest.append(business["name"])
    all_review.append(business["review_count"]) 
    all_rating.append(business["rating"])
    all_add.append(business["location"]["address1"]) 
    all_city.append(business["location"]["city"]) 

 

In [6]:
#Combine data
final_yelp = all_rest + all_add + all_city + all_rating + all_review


In [7]:
#put into DataFrames
all_rest_df = pd.DataFrame(all_rest)
all_rating_df = pd.DataFrame(all_rating) 
all_add_df = pd.DataFrame(all_add) 
all_city_df = pd.DataFrame(all_city) 
all_review_df = pd.DataFrame(all_review) 

In [9]:
#merge all dataframes and rename columns
merge1 = all_rest_df.merge(all_add_df, how='outer', left_index=True, right_index=True)
merge1 = merge1.rename(columns={"0_x": "Name"})
merge1 = merge1.rename(columns={"0_y": "Address"})
merge2 = merge1.merge(all_city_df, how='outer', left_index=True, right_index=True)
merge2 = merge2.rename(columns={"0": "City"})
merge3 = merge2.merge(all_rating_df, how='outer', left_index=True, right_index=True)
final_merge = merge3.merge(all_review_df, how='outer', left_index=True, right_index=True)
final_merge = final_merge.rename(columns={"Name": "Restaurant Name"})
final_merge = final_merge.rename(columns={"Address": "Address"})
final_merge = final_merge.rename(columns={"0_x": "City"})
final_merge = final_merge.rename(columns={"0_y": "Rating"})
final_merge.head()

,Restaurant Name,Address,City,Rating,0
0,Of Aire,3524 Sunset Blvd,Los Angeles,5.0,4
1,Citizen Mustard,None,Los Angeles,4.5,10
2,Elderberries Threefold Cafe,7564 W Sunset Blvd,Los Angeles,4.0,368
3,Gracias Madre,8905 Melrose Ave,West Hollywood,4.0,3662
4,Green Table Cafe,5998 W Pico Blvd,Los Angeles,4.5,375


In [ ]:
#rename using placement as column title has no name
final_merge.columns.values[4] = 'Reviewers Count'
final_merge

In [21]:
# append both dataframes

final_etl_data = veg_data_df.append(final_merge, ignore_index =False, sort=True)

final_etl_data

,0,Address,City,Rating,Restaurant Name,Reviewers Count
0,NaN,"Third Floor, Century City Mall, Kalayaan Avenu...",Makati City,4.8,Le Petit Souffle,314.0
1,NaN,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...",Makati City,4.5,Izakaya Kikufuji,591.0
2,NaN,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...",Mandaluyong City,4.4,Heat Edsa ShangriLa,270.0
3,NaN,"Third Floor, Mega Fashion Hall, SM Megamall, O...",Mandaluyong City,4.9,Ooma,365.0
4,NaN,"Third Floor, Mega Atrium, SM Megamall, Ortigas...",Mandaluyong City,4.8,Sambo Kojin,229.0
5,NaN,"Ground Floor, Mega Fashion Hall, SM Megamall, ...",Mandaluyong City,4.4,Din Tai Fung,336.0
6,NaN,"Building K, SM By The Bay, Sunset Boulevard, M...",Pasay City,4.0,Buffet,520.0
7,NaN,"Building B, By The Bay, Seaside Boulevard, Mal...",Pasay City,4.2,Vikings,677.0
8,NaN,"Plaza Level, Sofitel Philippine Plaza Manila, ...",Pasay City,4.9,Spiral Sofitel Philippine Plaza Manila,621.0
9,NaN,"Brixton Technology Center, 10 Brixton Street, ...",Pasig City,4.8,Locavore,532.0
